<a href="https://colab.research.google.com/github/atolman01/Sentiment_Analysis_Manning/blob/main/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Neural Networks** *are various algorithms that mimic the operation of the human brain to recognize relationships through vast amounts of data*

    A model based on neural networks utilizes a training set to learn and gain insights

**Install Requirements**

In [ ]:
#set up environment from crow_intelligence
!wget https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
!pip install -r requirements.txt

--2021-01-14 16:44:21--  https://raw.githubusercontent.com/crow-intelligence/growth-hacking-sentiment/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     151  --.-KB/s    in 0s      

2021-01-14 16:44:22 (10.3 MB/s) - ‘requirements.txt’ saved [151/151]

     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 3.2MB 13.7MB/s 
     |████████████████████████████████| 1.8MB 15.9MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 7.5MB 33.8MB/s 
     |████████████████████████████████| 1.9MB 59.3MB/s 
     |████████████████████████████████| 317kB 50.9MB/s 
     |████

**Install apex**


In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 7872, done.
remote: Total 7872 (delta 0), reused 0 (delta 0), pack-reused 7872
Receiving objects: 100% (7872/7872), 13.98 MiB | 29.21 MiB/s, done.
Resolving deltas: 100% (5374/5374), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-eyfsymv1
Created temporary directory: /tmp/pip-req-tracker-zlnxurrn
Created requirements tracker '/tmp/pip-req-tracker-zlnxurrn'
Created temporary directory: /tmp/pip-install-5hsqehpg
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-8xeo6f6o
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-zlnxurrn'
    Running setup.py (path:/tmp/pip-req-build-8xeo6f6o/setup.py) egg_info for package from file:///content/apex
    Runnin

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g5erL1CY9ENm7tJtjJOfaUFdOIQW4XMo95rZyvoBsMdFtHbbBBRsuA
Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install torch==1.5.0+cpu torchvision==0.6.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
import pandas as pd
import numpy as np

from transformers import pipeline,BertModel,BertTokenizer

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
#read in raw data
with open ("/content/drive/My Drive/NLP Python/Data/small_corpus.csv", 'r') as read_file:
  sm_corpus = pd.read_csv(read_file)

# only retrieve review and rating columns
reviews = list(sm_corpus['reviews'])
ratings = list(sm_corpus['ratings'])

df = pd.DataFrame({
    "rating" : ratings,
    "review" : reviews
})

#df.loc[2000,'rating']

3.0

In [ ]:
df.isnull().any()

rating    False
review    False
dtype: bool

In [ ]:
nlp = pipeline("sentiment-analysis")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# use the pipeline to classify review as positive or negative
# if labeled negative and score is greater than threshold --> negative
# if labeled positive and score is greater than threshold --> positive
# otherwise score is lower than threshold --> neutral

import gensim
import nltk
from gensim.parsing.preprocessing import remove_stopwords

nltk.download('punkt',quiet=True)
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords

def classify_sentiment(text):
  #the maximum sequence length is 512
  if len(text) > 512:
    #truncate the long review
    text = text[:512]
  result = nlp(text,model='bert-base-uncased',tokenizer='bert-base-uncased')
  if result[0]['label'] == 'NEGATIVE' and result[0]['score'] > 0.75:
    return -1
  elif result[0]['label'] == 'POSITIVE' and result[0]['score']  > 0.75:
    return 1
  else:
    return 0
#remove all stopwords from review
def remove_allstopwords(sentence):
  gensim_stopwords = gensim.parsing.preprocessing.STOPWORDS
  nltk_stopwords = stopwords.words('english')
  tokens = sentence.split(" ")
  #tokens_filtered= [word for word in lowered_tokens if not word in gensim_stopwords]
  tokens_filtered= [word for word in tokens if not word in nltk_stopwords]
  return (" ").join(tokens_filtered)

#because the maximum sequence of the pipeline is only 512
#must cut some of the reviews to 512
def fix_length_review(text):
  return text[:512] if len(text) > 512 else text


In [ ]:
review_no_swords = [remove_allstopwords(r) for r in df['review']]
df['review_truncate'] = [fix_length_review(r) for r in review_no_swords]


#df.head()
sentiment_classes = df['review_truncate'].apply(classify_sentiment)
sentiment_classes


0      -1
1      -1
2      -1
3      -1
4      -1
       ..
4495    1
4496    1
4497   -1
4498    1
4499    1
Name: review_truncate, Length: 4500, dtype: int64

In [ ]:
def classify_rating(rating):
  if rating in [4,5]:
    return 1
  elif rating == 3:
    return 0
  elif rating in [1,2]:
    return -1
rating_classes = [classify_rating(r) for r in ratings]
target_names = ["negative","neutral","positive"]

In [ ]:
# TODO add sentiment classes and rating classes to dataframe
# TODO clean up work from earlier code
from sklearn.metrics import classification_report, accuracy_score

report = classification_report(rating_classes,sentiment_classes)
print(report)
print(accuracy_score(rating_classes,sentiment_classes))


              precision    recall  f1-score   support

          -1       0.73      0.84      0.78      2000
           0       0.20      0.08      0.11       500
           1       0.78      0.78      0.78      2000

    accuracy                           0.73      4500
   macro avg       0.57      0.57      0.56      4500
weighted avg       0.69      0.73      0.71      4500

0.7291111111111112


In [ ]:
df['rating_class'] = rating_classes
df['sentiment_class'] = sentiment_classes

df.head()

,rating,review,review_truncate,rating_class,sentiment_class
0,1.0,"At first glance, this Sega Classics Collection...","At first glance, Sega Classics Collection appe...",-1,-1
1,1.0,Stopped working after a week. You get what you...,Stopped working week. You get pay for... Doesn...,-1,-1
2,1.0,"Cheap deaths, horrible camera, and no knowledg...","Cheap deaths, horrible camera, knowledge scien...",-1,-1
3,1.0,O.K. I will keep it short and sweet. I thoug...,O.K. I keep short sweet. I thought idea brus...,-1,-1
4,1.0,this Adapter would have been just fine....if i...,Adapter would fine....if actually worked.\nas ...,-1,-1


In [ ]:
from sklearn.model_selection import train_test_split

def encode_rating(rating):
  #negative = 0
  if rating == -1:
    return 0
  #neutral = 1
  elif rating == 0:
    return 1
  #positive = 2
  elif rating == 1:
    return 2

df['rating_class'] = df['rating_class'].apply(encode_rating)
  
#get train and test sets, t1 = train, t2 = test
t1_df, t2_df = train_test_split(df,stratify=df['rating_class'],random_state=42)

# because the model trained looks for text and label columns
# make a training dataframe with only those two columns
texts = t1_df['review']
labels = t1_df['rating_class']

train_df = pd.DataFrame({
    'text' : texts,
    'labels' : labels
})
train_df.head()

# do the same for the testing set, t1_df
texts_test = t2_df['review']
labels_test=t2_df['rating_class']

test_df = pd.DataFrame({
    'text' : texts_test,
    'labels' : labels_test
})

test_df.head()


,text,labels
127,Please save yourself the trouble don't buy thi...,0
1382,"I bought this for my fiance, she left me and I...",0
3093,Grandson will love,2
2128,"I thought long and hard before I picked up ""Li...",1
563,Keeps falling,0


In [ ]:
from simpletransformers.classification import ClassificationModel
# create the classification model using distilbert
classifier = ClassificationModel(model_type='distilbert',
                            model_name='distilbert-base-uncased',
                            num_labels=3,
                            use_cuda = False,
                            args={ 
                              'max_seq_length': 128,
                              'num_train_epochs': 10,
                              'output_dir': 'model_dir/',
                              'best_model_dir': 'model_dir/',
                              'evaluate_during_training': True,
                              'num_training_epochs': 20,
                              'train_batch_size': 20,
                              'eval_batch_size': 20
                            })

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
classifier.train_model(train_df=train_df,eval_df=test_df)

  0%|          | 0/3375 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/169 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 10:   0%|          | 0/169 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/169 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

https://huggingface.co/transformers/task_summary.html#sequence-classification

https://huggingface.co/transformers/main_classes/pipelines.html